# The Optimiser Interface

## Interacting with optimisers in PyBOP

This notebook introduces two interfaces to interact with PyBOP's optimiser classes.

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
import json

import numpy as np
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

## Setting up the problem

The code block below sets up the model, problem, and cost objects. For more information on this process, take a look at other notebooks in the examples directory.

In [ ]:
# Define the model
model = pybamm.equivalent_circuit.Thevenin(options={"number of rc elements": 1})

# Load the parameters
with open("../../parameters/initial_ecm_parameters.json") as file:
    parameter_values = pybamm.ParameterValues(json.load(file))
parameter_values.update(
    {
        "Open-circuit voltage [V]": model.default_parameter_values[
            "Open-circuit voltage [V]"
        ]
    },
    check_already_exists=False,
)

# Generate synthetic data
t_eval = np.arange(0, 900, 2)
sol = pybamm.Simulation(model, parameter_values=parameter_values).solve(t_eval=t_eval)

# Form dataset
dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": sol["Current [A]"](t_eval),
        "Voltage [V]": sol["Voltage [V]"](t_eval),
    }
)

# Define the fitting parameter
parameter_values.update(
    {
        "R0 [Ohm]": pybop.Parameter(
            prior=pybop.Gaussian(0.0002, 0.0001),
            bounds=[1e-4, 1e-2],
        )
    }
)

# Construct problem and cost
simulator = pybop.pybamm.Simulator(
    model,
    parameter_values=parameter_values,
    protocol=dataset,
)
cost = pybop.SumSquaredError(dataset)
problem = pybop.Problem(simulator, cost)

## Interacting with the optimisers

Now that we have set up the required objects, we can introduce the interface for interacting with the optimisers. In this example, we will see how to use PINTS optimisers and SciPy optimisers. Here is the complete [list of optimisers](https://pybop-docs.readthedocs.io/en/latest/optimisers.html) supported in PyBOP.

In the following example, PINTS-based optimiser set-up is shown. Optimiser arguments can be passed using PintsOptions class.

In [ ]:
options = pybop.PintsOptions(
    max_iterations=50,
    max_unchanged_iterations=25,
    absolute_tolerance=1e-9,
)
optim_one = pybop.XNES(
    problem, options=options
)  # Direct optimiser class with options object
optim_one.set_max_iterations(
    50
)  # Alternative set() / get() methods for PINTS optimisers
result = optim_one.run()
print(result)

OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [0.00031825]
  Optimised parameters: [0.00100003]
  Best cost: 1.774687256238677e-11
  Optimisation time: 0.8066089153289795 seconds
  Number of iterations: 45
  Number of evaluations: 178
  Reason for stopping: No significant change for 25 iterations.


Next, the use of SciPy optimisers is demonstrated. The `ScipyMinimizeOptions` class is used to pass the optimiser arguments.

In [ ]:
options = pybop.SciPyMinimizeOptions(
    maxiter=50,
    method="Nelder-Mead",
    tol=1e-9,
)
optim_two = pybop.SciPyMinimize(problem, options=options)
result2 = optim_two.run()
print(result2)

OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [0.00020912]
  Optimised parameters: [0.001]
  Best cost: 8.90154418583812e-12
  Optimisation time: 0.5370280742645264 seconds
  Number of iterations: 25
  Number of evaluations: 51
  Reason for stopping: Optimization terminated successfully.


Here is the optimisation results for Pints and SciPy optimisers.

In [ ]:
print("Estimated parameters x1:", result.x)
print("Estimated parameters x2:", result2.x)

Estimated parameters x1: [0.00100003]
Estimated parameters x2: [0.001]
